# Основной пример AutoGen

В этом примере кода вы будете использовать [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework для создания базового агента.

Цель этого примера — показать вам шаги, которые мы позже будем использовать в дополнительных примерах кода при реализации различных агентных шаблонов.


## Импорт необходимых пакетов Python


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Создание клиента

В этом примере мы будем использовать [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступа к LLM.

`model` определен как `gpt-4o-mini`. Попробуйте изменить модель на другую, доступную на платформе GitHub Models, чтобы увидеть разные результаты.

Для быстрого теста мы просто запустим простой запрос — «Какая столица Франции».


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Определение агента

Теперь, когда мы настроили `client` и убедились, что он работает, давайте создадим `AssistantAgent`. Каждому агенту можно назначить:  
**name** - Короткое имя, которое будет полезно для ссылки на него в сценариях с несколькими агентами.  
**model_client** - Клиент, который вы создали на предыдущем этапе.  
**tools** - Доступные инструменты, которые агент может использовать для выполнения задачи.  
**system_message** - Метаподсказка, определяющая задачу, поведение и тон LLM.  

Вы можете изменить системное сообщение, чтобы увидеть, как LLM реагирует. Мы рассмотрим `tools` в Уроке №4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Запуск агента

Ниже приведена функция для запуска агента. Мы используем метод `on_message`, чтобы обновить состояние агента новым сообщением.

В данном случае мы обновляем состояние новым сообщением от пользователя: `"Спланируй мне отличный солнечный отпуск"`.

Вы можете изменить содержание сообщения, чтобы увидеть, как LLM реагирует по-разному.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
